In [1]:
import pandas as pd
import os
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from rank_bm25 import BM25Okapi
from sentence_transformers import SentenceTransformer, util
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from tqdm import tqdm


/mnt/c/Users/sasidhar.chennup/Documents/Tiger-Training/NLP-GenAI-Classroom/NLP-Gen-AI-classroom/.nlp-genai/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
%pwd

'/mnt/c/Users/sasidhar.chennup/Documents/Tiger-Training/NLP-GenAI-Classroom/NLP-Gen-AI-classroom/Assignment-2'

In [3]:
%cd /mnt/c/Users/sasidhar.chennup/Documents/Tiger-Training/NLP-GenAI-Classroom/NLP-Gen-AI-classroom


/mnt/c/Users/sasidhar.chennup/Documents/Tiger-Training/NLP-GenAI-Classroom/NLP-Gen-AI-classroom


/mnt/c/Users/sasidhar.chennup/Documents/Tiger-Training/NLP-GenAI-Classroom/NLP-Gen-AI-classroom/.nlp-genai/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [5]:
#Loading data sets
qa_df = pd.read_csv("data/squad_v2_dataset.csv", header=None, names=["question", "answers", "context_id"],encoding="latin1")
ctx_df = pd.read_csv("data/context.csv", header=None, names=["context_id", "context"],encoding="latin1")


In [ ]:

contexts = ctx_df["context"].tolist()
context_ids = ctx_df["context_id"].tolist()

# Build TF-IDF
tfidf_vectorizer = TfidfVectorizer(stop_words="english")
tfidf_matrix = tfidf_vectorizer.fit_transform(contexts)

# Build BM25
bm25=BM25Okapi(contexts)

def bm25_retrieve(query,top_k=10):
    scores=bm25.get_scores(query)
    top_index=np.argsort(scores)[::-1][:top_k]
    return [context_ids[i] for i in top_index],scores[top_index]


def retrieve_tfidf(query, top_k=5):
    q_vec = tfidf_vectorizer.transform([query])
    scores = cosine_similarity(q_vec, tfidf_matrix).ravel()
    top_idx = np.argsort(scores)[::-1][:top_k]
    return [context_ids[i] for i in top_idx], scores[top_idx]

def recall_at_k(retrieved_ids, gold_id, k):
    return 1 if gold_id in retrieved_ids[:k] else 0

def reciprocal_rank_at_k(retrieved_ids, gold_id, k):
    if gold_id in retrieved_ids[:k]:
        rank = retrieved_ids.index(gold_id) + 1
        return 1.0 / rank
    return 0.0

def ndcg_at_k(retrieved_ids, gold_id, k):
    if gold_id not in retrieved_ids[:k]:
        return 0.0
    rank = retrieved_ids.index(gold_id) + 1
    dcg = 1 / np.log2(rank + 1)
    idcg = 1.0  # ideal DCG if correct doc at rank 1
    return dcg / idcg



100%|██████████| 11819/11819 [01:29<00:00, 131.85it/s]


tf-idf_Recall@1: 0.5938
tf-idf_Recall@3: 0.7629
tf-idf_Recall@5: 0.8156
tf-idf_Recall@10: 0.8685
tf-idf_Recall@20: 0.9000
tf-idf_MRR@1: 0.5938
tf-idf_MRR@3: 0.6701
tf-idf_MRR@5: 0.6822
tf-idf_MRR@10: 0.6894
tf-idf_MRR@20: 0.6917
tf-idf_NDCG@1: 0.5938
tf-idf_NDCG@3: 0.6940
tf-idf_NDCG@5: 0.7158
tf-idf_NDCG@10: 0.7331
tf-idf_NDCG@20: 0.7411
bm25_NDCG@1: 0.0102
bm25_NDCG@3: 0.0180
bm25_NDCG@5: 0.0231
bm25_NDCG@10: 0.0294
bm25_NDCG@20: 0.0548
bm25_Recall@1: 0.0102
bm25_Recall@3: 0.0237
bm25_Recall@5: 0.0360
bm25_Recall@10: 0.0558
bm25_Recall@20: 0.1558
bm25_MRR@1: 0.0102
bm25_MRR@3: 0.0160
bm25_MRR@5: 0.0189
bm25_MRR@10: 0.0214
bm25_MRR@20: 0.0285


In [ ]:
# --- Evaluation ---
def evaluate_retriever(qa_df, k_values=[1,3,5,10,20]):
    results = {f"tf-idf_Recall@{k}": [] for k in k_values}
    results.update({f"tf-idf_MRR@{k}": [] for k in k_values})
    results.update({f"tf-idf_NDCG@{k}": [] for k in k_values})
    results.update({f"bm25_NDCG@{k}": [] for k in k_values})
    results.update({f"bm25_Recall@{k}": [] for k in k_values})
    results.update({f"bm25_MRR@{k}": [] for k in k_values})

    for _, row in tqdm(qa_df.iterrows(), total=len(qa_df)):
        query = row["question"]
        gold_id = row["context_id"]

        retrieved_ids, _ = retrieve_tfidf(query, top_k=max(k_values))
        bm_retrieveids,scores_bm25= bm25_retrieve(query,top_k=max(k_values))
        #print(f"bmretrieves:{bm_retrieveids},goldid:{gold_id}")

        for k in k_values:
            results[f"tf-idf_Recall@{k}"].append(recall_at_k(retrieved_ids, gold_id, k))
            results[f"tf-idf_MRR@{k}"].append(reciprocal_rank_at_k(retrieved_ids, gold_id, k))
            results[f"tf-idf_NDCG@{k}"].append(ndcg_at_k(retrieved_ids, gold_id, k))
            results[f"bm25_Recall@{k}"].append(recall_at_k(bm_retrieveids,gold_id,k))
            results[f"bm25_MRR@{k}"].append(reciprocal_rank_at_k(bm_retrieveids, gold_id, k))
            results[f"bm25_NDCG@{k}"].append(ndcg_at_k(bm_retrieveids, gold_id, k))

    # Average results
    return {metric: np.mean(vals) for metric, vals in results.items()}

# Example usage
metrics = evaluate_retriever(qa_df, k_values=[1,3,5,10,20])
for k, v in metrics.items():
    print(f"{k}: {v:.4f}")

In [32]:
qa_df.head()

,question,answers,context_id
0,In what country is Normandy located?,"{'text': ['France', 'France', 'France', 'Franc...",0
1,When were the Normans in Normandy?,"{'text': ['10th and 11th centuries', 'in the 1...",0
2,From which countries did the Norse originate?,"{'text': ['Denmark, Iceland and Norway', 'Denm...",0
3,Who was the Norse leader?,"{'text': ['Rollo', 'Rollo', 'Rollo', 'Rollo'],...",0
4,What century did the Normans first gain their ...,"{'text': ['10th century', 'the first half of t...",0


In [33]:
ctx_df.head()

,context_id,context
0,0,The Normans (Norman: Nourmands; French: Norman...
1,1,"The Norman dynasty had a major political, cult..."
2,2,"The English name ""Normans"" comes from the Fren..."
3,3,"In the course of the 10th century, the initial..."
4,4,"Before Rollo's arrival, its populations did no..."


In [34]:
qa_df["context_id"] = qa_df["context_id"].astype(str)
ctx_df["context_id"] = ctx_df["context_id"].astype(str)

In [35]:
#Preprocessing Function
def normalize_text(text):
    text = text.lower()
    text = re.sub(r"\s+", " ", text)      # remove extra spaces
    text = re.sub(r"[^\w\s]", "", text)   # remove punctuation
    return text.strip()

qa_df["question_norm"] = qa_df["question"].apply(normalize_text)
ctx_df["context_norm"] = ctx_df["context"].apply(normalize_text)

In [36]:
qa_df.head()

,question,answers,context_id,question_norm
0,In what country is Normandy located?,"{'text': ['France', 'France', 'France', 'Franc...",0,in what country is normandy located
1,When were the Normans in Normandy?,"{'text': ['10th and 11th centuries', 'in the 1...",0,when were the normans in normandy
2,From which countries did the Norse originate?,"{'text': ['Denmark, Iceland and Norway', 'Denm...",0,from which countries did the norse originate
3,Who was the Norse leader?,"{'text': ['Rollo', 'Rollo', 'Rollo', 'Rollo'],...",0,who was the norse leader
4,What century did the Normans first gain their ...,"{'text': ['10th century', 'the first half of t...",0,what century did the normans first gain their ...


In [37]:
ctx_df.head()

,context_id,context,context_norm
0,0,The Normans (Norman: Nourmands; French: Norman...,the normans norman nourmands french normands l...
1,1,"The Norman dynasty had a major political, cult...",the norman dynasty had a major political cultu...
2,2,"The English name ""Normans"" comes from the Fren...",the english name normans comes from the french...
3,3,"In the course of the 10th century, the initial...",in the course of the 10th century the initiall...
4,4,"Before Rollo's arrival, its populations did no...",before rollos arrival its populations did not ...


In [38]:
contexts = ctx_df["context_norm"].tolist()
querys=qa_df['question_norm'].tolist()
context_ids = ctx_df["context_id"].tolist()

In [39]:
contexts

['the normans norman nourmands french normands latin normanni were the people who in the 10th and 11th centuries gave their name to normandy a region in france they were descended from norse norman comes from norseman raiders and pirates from denmark iceland and norway who under their leader rollo agreed to swear fealty to king charles iii of west francia through generations of assimilation and mixing with the native frankish and romangaulish populations their descendants would gradually merge with the carolingianbased cultures of west francia the distinct cultural and ethnic identity of the normans emerged initially in the first half of the 10th century and it continued to evolve over the succeeding centuries',
 'the norman dynasty had a major political cultural and military impact on medieval europe and even the near east the normans were famed for their martial spirit and eventually for their christian piety becoming exponents of the catholic orthodoxy into which they assimilated th

In [40]:
context_ids

['0',
 '1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9',
 '10',
 '11',
 '12',
 '13',
 '14',
 '15',
 '16',
 '17',
 '18',
 '19',
 '20',
 '21',
 '22',
 '23',
 '24',
 '25',
 '26',
 '27',
 '28',
 '29',
 '30',
 '31',
 '32',
 '33',
 '34',
 '35',
 '36',
 '37',
 '38',
 '39',
 '40',
 '41',
 '42',
 '43',
 '44',
 '45',
 '46',
 '47',
 '48',
 '49',
 '50',
 '51',
 '52',
 '53',
 '54',
 '55',
 '56',
 '57',
 '58',
 '59',
 '60',
 '61',
 '62',
 '63',
 '64',
 '65',
 '66',
 '67',
 '68',
 '69',
 '70',
 '72',
 '73',
 '76',
 '78',
 '79',
 '80',
 '81',
 '82',
 '83',
 '84',
 '85',
 '86',
 '87',
 '88',
 '89',
 '90',
 '91',
 '92',
 '93',
 '94',
 '95',
 '97',
 '98',
 '99',
 '100',
 '101',
 '102',
 '103',
 '104',
 '105',
 '106',
 '107',
 '108',
 '109',
 '110',
 '111',
 '112',
 '113',
 '114',
 '115',
 '116',
 '117',
 '118',
 '119',
 '120',
 '121',
 '122',
 '123',
 '124',
 '125',
 '126',
 '127',
 '128',
 '129',
 '130',
 '131',
 '132',
 '133',
 '134',
 '135',
 '136',
 '137',
 '138',
 '139',
 '140',
 '141',
 '142',
 

In [67]:
tfidf_vectorizer = TfidfVectorizer(stop_words=None)

In [ ]:
# print(ctx_df['context'].isna().sum())
# # Sanity check: look for empty / whitespace contexts
# empty_contexts = [i for i, c in enumerate(contexts) if not c.strip()]
# print(f"Total contexts: {len(contexts)}")
# print(f"Empty or whitespace-only contexts: {len(empty_contexts)}")

# # Preview first few empty ones
# if empty_contexts:
#     print("Examples of empty contexts:")
#     for idx in empty_contexts[:5]:
#         print(f"Index {idx} -> '{contexts[idx]}'")

# # Check very short contexts (len < 5 chars)
# short_contexts = [i for i, c in enumerate(contexts) if len(c.split()) < 3]
# print(f"Very short contexts (<3 words): {len(short_contexts)}")

# # Check vocabulary size after TF-IDF fit
# from sklearn.feature_extraction.text import TfidfVectorizer
# vectorizer = TfidfVectorizer(stop_words=None)  # no stopword removal yet
# tfidf_matrix = vectorizer.fit_transform(contexts)

# print("TF-IDF matrix shape:", tfidf_matrix.shape)
# print("Vocabulary size:", len(vectorizer.vocabulary_))

# # Check if any row is all zeros
# import numpy as np
# row_sums = np.array(tfidf_matrix.sum(axis=1)).ravel()
# zero_rows = np.where(row_sums == 0)[0]
# print(f"Contexts with all-zero TF-IDF vectors: {len(zero_rows)}")
# if len(zero_rows) > 0:
#     print("Example indices:", zero_rows[:10])


0
Total contexts: 1120
Empty or whitespace-only contexts: 0
Very short contexts (<3 words): 0
TF-IDF matrix shape: (1120, 17021)
Vocabulary size: 17021
Contexts with all-zero TF-IDF vectors: 0


In [68]:
tfidf_matrix = tfidf_vectorizer.fit_transform(contexts+[querys[0]])

In [70]:
query_vector = tfidf_matrix[-1]  # last row is query
context_vectors = tfidf_matrix[:-1]  # all contexts
print(tfidf_matrix.shape)
# print(tfidf_matrix.toarray().sum())
# print(question_vector)
# print(tfidf_matrix.toarray())

# query_vector = tfidf_matrix.toarray()[0]
# nonzero_indices = np.nonzero(query_vector)[0]
# print("Number of non-zero TF-IDF values in query:", len(nonzero_indices))
# print("Indices of first 10 non-zero terms:", nonzero_indices[:10])
# print("TF-IDF values of first 10 non-zero terms:", query_vector[nonzero_indices[:10]])

(1121, 17021)


In [ ]:
# print('\nidf values:')
# for ele1, ele2 in zip(tfidf_vectorizer.get_feature_names_out(), tfidf_vectorizer.idf_):
#     print(ele1, ':', ele2)


idf values:
0079 : 7.328829242512215
01 : 6.635682061952269
01250 : 7.328829242512215
02 : 6.92336413440405
03 : 7.328829242512215
037 : 7.328829242512215
039118 : 7.328829242512215
04 : 6.92336413440405
05 : 6.92336413440405
0514 : 7.328829242512215
052sq : 7.328829242512215
06 : 7.328829242512215
062 : 7.328829242512215
066 : 7.328829242512215
07 : 6.92336413440405
08 : 7.328829242512215
09 : 6.92336413440405
095 : 7.328829242512215
10 : 4.88648220714301
100 : 5.249387700832378
1000 : 5.942534881392324
10000 : 6.230216953844105
100000 : 5.719391330078114
1000000 : 7.328829242512215
10006721 : 7.328829242512215
100106 : 7.328829242512215
100150 : 7.328829242512215
1010 : 7.328829242512215
1011 : 6.92336413440405
1012 : 6.92336413440405
1013 : 6.635682061952269
1015 : 6.635682061952269
1016 : 7.328829242512215
1017 : 7.328829242512215
1018 : 7.328829242512215
102 : 7.328829242512215
1024bit : 7.328829242512215
10311095 : 7.328829242512215
103620 : 7.328829242512215
1038 : 7.3288292425

In [ ]:
similarities = cosine_similarity(query_vector, context_vectors).flatten()
top_indices = similarities.argsort()[::-1][:5]
print("Top 5 similar context indices:", top_indices)
print("Similarity scores:", similarities[top_indices])



Top 5 similar context indices: [ 14  33   6 500  16]
Similarity scores: [0.22595288 0.14204779 0.12879889 0.12387775 0.12367669]
